# Toolhouse with Ollama

Now let's set up the Jupyter environment to handle asynchronous programming and enables the display of rich HTML content.

1. **`import nest_asyncio`**: Imports the `nest_asyncio` library, which allows for the smooth handling of asynchronous tasks in environments like Google Colab, where nested event loops might otherwise cause issues.

2. **`nest_asyncio.apply()`**: Applies a patch that enables running asynchronous code without conflicts. This is particularly useful when working with `asyncio` in notebooks, allowing for proper execution of asynchronous tasks.

3. **`from IPython.display import display, HTML`**: Imports functions from IPython’s display module that allow for the rendering of rich media content (like HTML) directly within the notebook. This enables the output of formatted HTML, which can be used for interactive content or visually enriched outputs.

This setup ensures that you can effectively run and display asynchronous code alongside HTML content in the same notebook.

In [1]:
import nest_asyncio

nest_asyncio.apply()

from IPython.display import display, HTML

Ensure you have Ollama installed

In [2]:
!which ollama

/usr/local/bin/ollama


To install Ollama: https://ollama.com/download

Install the ollama package from the Python Package Index (PyPI)

In [3]:
%pip install ollama --quiet

Note: you may need to restart the kernel to use updated packages.


Ensure you have Mistral in Ollama

In [4]:
!ollama list

NAME          	ID          	SIZE  	MODIFIED     
mistral:latest	f974a74358d6	4.1 GB	46 hours ago	
llama3:latest 	365c0bd3c000	4.7 GB	2 months ago	


To install Mistral on Ollama run the following cell

In [5]:
!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling 491dfa501e59... 100% ▕████████████████▏  801 B                         
pulling ed11eda7790d... 100% ▕████████████████▏   30 B                         
pulling 42347cd80dc8... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest 

In [6]:
from getpass import getpass

# Prompt the user to enter the Exa API key securely
EXA_API_KEY = getpass("Enter your Exa API key: ")

Test the Exa AI search function using a direct web search query.

In [7]:
import json
import ollama
import asyncio
import requests

# Web search using Exa AI
def search_web(query: str) -> str:
  headers = {
    'accept': 'application/json',
    'content-type': 'application/json',
    'x-api-key': EXA_API_KEY,
  }
  data = {
    "query": query,
    "type": "neural",
    "useAutoprompt": True,
    "numResults": 10,
    "contents": {
      "text": True
    }
  }
  # Correct Exa AI search endpoint
  response = requests.post('https://api.exa.ai/search', headers=headers, json=data)
  
  # Check if the response is valid JSON
  try:
    response.raise_for_status()  # Raise an error for bad status codes
    response_json = response.json()
  except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    return json.dumps({'error': 'HTTP error occurred'})
  except json.JSONDecodeError:
    print("Failed to decode JSON. Response content:")
    print(response.text)
    return json.dumps({'error': 'Failed to decode JSON response'})

  return json.dumps(response_json)

async def run(model: str):
  client = ollama.AsyncClient()
  # Initialize conversation with a user query
  messages = [{'role': 'user', 'content': 'Search the web for "current weather in New York"'}]

  # First API call: Send the query and function description to the model
  response = await client.chat(
    model=model,
    messages=messages,
    tools=[
      {
        'type': 'function',
        'function': {
          'name': 'search_web',
          'description': 'Search the web for a given query',
          'parameters': {
            'type': 'object',
            'properties': {
              'query': {
                'type': 'string',
                'description': 'The search query',
              },
            },
            'required': ['query'],
          },
        },
      },
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided function
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # Process function calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'search_web': search_web,
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['query'])
      # Add function response to the conversation
      messages.append(
        {
          'role': 'tool',
          'content': function_response,
        }
      )

  # Second API call: Get final response from the model
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])


# Run the async function
asyncio.run(run('mistral'))

1. Title: Articles about hot | Gothamist
    - URL: https://gothamist.com/tags/hot
    - Published Date: 2021-07-06T00:00:00.000Z
    - Author: Jen Chung
    - Text Extracted: "NYC is under heat advisory from 11 a.m. on Tuesday until 8 p.m. on Wednesday. We're talking in the mid-90s, but with a real feel of melting in place. It will feel close to 100 degrees with humidity. It's not the REAL unofficial start of summer until the humidity hits. Temperatures are expected to hit in the mid-90s Saturday, Sunday and Monday, and the heat index is expected to be above 100."

2. Title: Articles about foggy | Gothamist
    - URL: https://gothamist.com/tags/foggy
    - Published Date: 2014-01-12T00:00:00.000Z
    - Author: Ben Yakas
    - Text Extracted: "Check out some gorgeous photos of the city enveloped in a thick coat of low-lying stratus clouds."


Install the Toolhouse package from the Python Package Index (PyPI)

In [8]:
%pip install toolhouse --quiet

Note: you may need to restart the kernel to use updated packages.


In [9]:
from getpass import getpass

# Prompt the user to enter the Exa API key securely
TH_API_KEY = getpass("Enter your ToolHouse API key: ")

Check available tools in ToolHouse store

In [16]:
# List available tools
available_tools = th.get_tools()
print("Available tools:")
for tool in available_tools:
    print(f"- {tool['function']['name']}")

Available tools:
- exa_web_search
- current_time
- pdf2csv


In [30]:
import nest_asyncio
import json
import ollama
import asyncio
from toolhouse import Toolhouse

nest_asyncio.apply()

# Initialize Toolhouse
th = Toolhouse(access_token=TH_API_KEY)

# Define the Ollama model we want to use
MODEL = 'mistral'

async def run_with_toolhouse(model: str):
    client = ollama.AsyncClient()
    # Initialize conversation with a user query
    messages = [{
        "role": "user",
        "content": "Search the web for current weather in New York using exa_web_search"
    }]

    # First API call: Send the query and function description to the model
    response = await client.chat(
        model=model,
        messages=messages,
        tools=th.get_tools(),  # This includes the Code Execution tool
    )

    print("Original response from Ollama:")
    print(json.dumps(response, indent=2))

    # Add the model's response to the conversation history
    messages.append(response['message'])

    # Check if the model decided to use the provided function
    if response['message'].get('tool_calls'):
        print("\nTool calls found in the response:")
        print(json.dumps(response['message']['tool_calls'], indent=2))

        # Adapt the response to match what Toolhouse expects
        adapted_response = {
            'choices': [{
                'message': response['message'],
                'finish_reason': 'tool_calls'
            }]
        }
        print("\nAdapted response for Toolhouse:")
        print(json.dumps(adapted_response, indent=2))

        # Process function calls made by the model
        try:
            tool_responses = th.run_tools(adapted_response)
            print("\nTool responses:")
            print(json.dumps(tool_responses, indent=2))
            messages.extend(tool_responses)
        except Exception as e:
            print(f"\nError running tools: {str(e)}")
            return

    # Second API call: Get final response from the model
    final_response = await client.chat(model=model, messages=messages)
    print("\nFinal response:")
    print(final_response['message']['content'])

# Run the async function
await run_with_toolhouse(MODEL)

Original response from Ollama:
{
  "model": "mistral",
  "created_at": "2024-09-06T13:32:34.951406Z",
  "message": {
    "role": "assistant",
    "content": "",
    "tool_calls": [
      {
        "function": {
          "name": "exa_web_search",
          "arguments": {
            "query": "current weather in New York"
          }
        }
      },
      {
        "function": {
          "name": "exa_web_search",
          "arguments": {
            "num_results": 5,
            "query": "current weather in New York"
          }
        }
      },
      {
        "function": {
          "name": "pdf2csv",
          "arguments": {
            "pdf_url": "<YOUR_PDF_URL>"
          }
        }
      }
    ]
  },
  "done_reason": "stop",
  "done": true,
  "total_duration": 3612544417,
  "load_duration": 14271834,
  "prompt_eval_count": 377,
  "prompt_eval_duration": 930490000,
  "eval_count": 149,
  "eval_duration": 2666123000
}

Tool calls found in the response:
[
  {
    "function": {